In [ ]:
# %%
import unicodedata
import re
from transformers import M2M100ForConditionalGeneration, M2M100Tokenizer
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.tools import DuckDuckGoSearchRun
from langchain.agents import initialize_agent, Tool
import google.generativeai as genai
import torch
from dotenv import load_dotenv
import os
import PyPDF2
from transformers import M2M100Tokenizer, M2M100ForConditionalGeneration
import sentencepiece as spm
from langchain.schema import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders import PyPDFLoader

from langchain_community.embeddings import HuggingFaceEmbeddings




c:\Users\HP\Desktop\project\Farm_9ja\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Access your keys

google_api_key = os.getenv("GOOGLE_API_KEY")

# Load environment variables
load_dotenv()

# Fetch key from .env
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")



print("Google Key:", google_api_key[:5] + "*")  # just to confirm it's loaded

Google Key: AIzaS*


In [3]:
# %%
# Normalization Function: preserves diacritics, cleans whitespace
def normalize_text(text):
    text = unicodedata.normalize('NFC', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text


In [4]:
# Post-processing: remove repeated phrases > max_repeat times
def clean_repetitions(text, max_repeat=2):
    pattern = r'(\b\w+\b(?: \b\w+\b){0,3})\s*(?:\1\s*){' + str(max_repeat) + ',}'
    cleaned_text = re.sub(pattern, r'\1', text, flags=re.IGNORECASE)
    return cleaned_text


In [5]:
# %%
# Load HelpMumHQ translation model & tokenizer
model_name = "HelpMumHQ/AI-translator-eng-to-9ja"
tokenizer = M2M100Tokenizer.from_pretrained(model_name)
translator = M2M100ForConditionalGeneration.from_pretrained(model_name).to("cuda" if torch.cuda.is_available() else "cpu")



In [6]:
# Translator function call using HelpMumHQ model
def translator_fn(text, src_lang, tgt_lang):
    tokenizer.src_lang = src_lang
    inputs = tokenizer(text, return_tensors="pt", padding=True).to(translator.device)
    generated_tokens = translator.generate(
        **inputs,
        forced_bos_token_id=tokenizer.get_lang_id(tgt_lang),
        max_new_tokens=512
    )
    translated = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0]
    return translated


In [7]:
# Define Gemini LLM wrapper
from langchain_google_genai import ChatGoogleGenerativeAI

# Use ChatGoogleGenerativeAI as the LLM
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", temperature=0.0, google_api_key=GOOGLE_API_KEY)

In [8]:
# LLM translator function using prompt-based translation
from langchain_core.messages import HumanMessage

def llm_translate(text, src_lang, tgt_lang):
    prompt_translate = f"Translate this text from {supported_langs.get(src_lang, src_lang)} to {supported_langs.get(tgt_lang, tgt_lang)}:\n\n{text}"
    # Call llm with HumanMessage
    response = llm([HumanMessage(content=prompt_translate)])
    return response.content

# %%
# Translation with fallback to LLM if output is missing or repetitive
def translate_with_fallback(text, src_lang, tgt_lang):
    normalized_text = normalize_text(text)

    try:
        translated = translator_fn(normalized_text, src_lang, tgt_lang)
        cleaned = clean_repetitions(translated)

        if len(cleaned.strip()) < 10 or cleaned == translated:
            # Fallback to LLM if output too short or repetitive
            fallback = llm_translate(normalized_text, src_lang, tgt_lang)
            return fallback
        else:
            return cleaned
    except Exception as e:
        # On error fallback to LLM
        return llm_translate(normalized_text, src_lang, tgt_lang)

In [ ]:


# Step 1: Load PDF
pdf_path = "data/best_practices_for_treating_crop_disease1.pdf"
loader = PyPDFLoader(pdf_path)
documents = loader.load()

# Step 2: Split PDF into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
docs = text_splitter.split_documents(documents)

# Step 3: Extract only the text content
texts = [doc.page_content for doc in docs]

# Filter out non-string and empty entries
texts = [t for t in texts if isinstance(t, str) and t.strip()]

# Step 4: Initialize embeddings
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Step 5: Create embeddings for the texts
embeddings = embedding_model.embed_documents(texts)
vectorstore = FAISS.from_texts(texts, embedding_model)
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k":3})

print(f"✅ Created {len(embeddings)} embeddings from {len(texts)} chunks")

✅ Created 9 embeddings from 9 chunks


In [16]:
# Prompt template for RAG answering
prompt_template = """
You are a friendly expert agricultural assistant for smallholder farmers.

Use the following context to answer the question comprehensively, clearly, and with practical advice.

Context:

{context}

Question:
{question}

Instructions:
- Use simple, non-technical language.
- Provide actionable tips farmers can immediately apply.
- Explain terms if complex vocabulary is necessary.
- If unsure, honestly say you do not know.

Answer:
"""

prompt = PromptTemplate(template=prompt_template, input_variables=["context", "question"])

qa_chain = RetrievalQA.from_chain_type(
    llm=llm, retriever=retriever, chain_type="stuff",
    return_source_documents=True, chain_type_kwargs={"prompt": prompt}
)

In [17]:
# DuckDuckGo search fallback tool
from duckduckgo_search import DDGS
ddg_search = DuckDuckGoSearchRun()
tools = [Tool(name="DuckDuckGo Search", func=ddg_search.run, description="Web search for farming topics")]


In [18]:
agent = initialize_agent(tools, llm, agent="zero-shot-react-description", verbose=True)

# %%
# Supported language codes
supported_langs = {
    'yo': 'yoruba',
    'ig': 'igbo',
    'ha': 'hausa',
    'en': 'english'
}


C:\Users\HP\AppData\Local\Temp\ipykernel_6876\3075973003.py:1: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(tools, llm, agent="zero-shot-react-description", verbose=True)


In [19]:
# Final function answering questions with translation, RAG, and fallback logic
def answer_question(user_question, user_lang):
    user_lang = user_lang if user_lang in supported_langs else 'en'

    # Translate question to English if needed
    if user_lang != 'en':
        query_en = translate_with_fallback(user_question, user_lang, 'en')
    else:
        query_en = user_question

    # Retrieve and generate answer
    result = qa_chain.invoke({"query": query_en})
    rag_answer = result['result']

    # Fallback to web search if answer is poor
    if len(rag_answer.strip()) < 10 or "I don't know" in rag_answer.lower():
        rag_answer = agent.run(query_en)

    # Translate answer back to user language if needed
    if user_lang != 'en':
        rag_answer = translate_with_fallback(rag_answer, 'en', user_lang)

    return rag_answer


In [20]:
# Example interactive usage
print("Supported languages: yo (Yoruba), ig (Igbo), ha (Hausa), en (English)")
user_lang_code = input("Enter language code: ").strip().lower()
user_question = input("Enter your question: ").strip()

final_answer = answer_question(user_question, user_lang_code)
print("\nAnswer:\n", final_answer)

Supported languages: yo (Yoruba), ig (Igbo), ha (Hausa), en (English)


C:\Users\HP\AppData\Local\Temp\ipykernel_6876\1065660348.py:7: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = llm([HumanMessage(content=prompt_translate)])



Answer:
 Dáadáa, ṣe o fẹ́ mọ bí a ṣe lè mú kí àwọn irúgbìn rẹ ní ìlera tí wọn kò sì ní àrùn? Ìbéèrè tó dára! Dídá àwọn àrùn dúró rọrùn ju gbígbìyànjú láti wo wọn sàn lẹ́yìn náà. Èyí ni bí o ṣe lè ṣe é:

**1. Mọ Ohun Tí O Ń Bá Ṣe:**

*   **Jẹ́ Aṣọ́jú Irúgbìn:** Kí o tó ṣe ohunkóhun, wo àwọn irúgbìn rẹ dáadáa. Ṣé àwọn àmì, ìrẹ̀wẹ̀sì, tàbí àwọn ohun àjèjì mìíràn ń ṣẹlẹ̀? Gbìyànjú láti mọ *ìdí* tí ó fi ń ṣẹlẹ̀. Ṣé àrùn ni, tàbí ó lè jẹ́ ohun mìíràn, bíi àìtó omi tàbí oúnjẹ? Mímọ ìyàtọ̀ ṣe pàtàkì!
*   Ronú nípa rẹ̀ bí èyí: *Àmì* dà bí rírí ọ̀daràn gangan (àrùn fúnra rẹ̀, bíi imú tàbí bakitéríà). *Àmì àìsàn* dà bí rírí ìpalára tí ọ̀daràn náà fà (bíi irúgbìn tí ó ń rẹ̀wẹ̀sì).
*   **Bí o kò bá dájú, béèrè fún ìrànlọ́wọ́!** Ọ́fíìsì ìtẹ́lọ́rùn àgbẹ̀ ti àdúgbò rẹ tàbí àgbẹ̀ kan tí ó ní ìmọ̀ lè ràn ọ́ lọ́wọ́ láti mọ ìṣòro náà.

**2. Mú Kí Àwọn Nǹkan Mọ́:**

*   **Mọ́ Lẹ́yìn Ìkórè:** Yọ àwọn ẹ̀yà irúgbìn àtijọ́ (ewé, igi, èso) lẹ́yìn tí o bá kórè. Àwọn wọ̀nyí lè jẹ́ ibi tí àwọn àrùn ti ń pọ̀ sí i.

In [21]:

# Example interactive usage
print("Supported languages: yo (Yoruba), ig (Igbo), ha (Hausa), en (English)")
user_lang_code = input("Enter language code: ").strip().lower()
user_question = input("Enter your question: ").strip()

final_answer = answer_question(user_question, user_lang_code)
print("\nAnswer:\n", final_answer)

Supported languages: yo (Yoruba), ig (Igbo), ha (Hausa), en (English)

Answer:
 Okay, this is a longer translation, so I'll do my best to capture the meaning and tone in Igbo. It's important to note that Igbo, like many languages, has regional variations, and the best translation might depend on the specific dialect. I'll aim for a generally understandable version.

**Igbo Translation:**

Ọ dị mma, aghọtara m na ị na-ajụ "Gịnị mere m ga-eji chọọ...?" mana ị kwughachiri ya ọtụtụ ugboro! Ka m wee nye gị azịza kacha mma, ọ dị mkpa ka m mara ihe ị na-ajụ. Agbanyeghị, dabere na ihe ọmụma m nwere gbasara ịkọ osikapa, echere m na ị nwere ike na-ajụ: **"Gịnị mere m ga-eji chọọ ịgbaso ndụmọdụ gbasara omenala ịkọ osikapa, ịdị ọcha, na njikwa ọrịa osikapa (Rice Blast)?"** Ọ bụrụ na ọ bụ ya, lee ihe mere ị ga-eji chọọ ime ihe ndị ahụ:

*   **Osisi Dị Mma, Owuwe Ihe Ubi Ka Ukwuu:** Chee echiche banyere ya otu a: osisi dị mma bụ osisi nwere obi ụtọ, osisi nwere obi ụtọ na-enyekwa gị osikapa karịa! Ị

In [22]:

# Example interactive usage
print("Supported languages: yo (Yoruba), ig (Igbo), ha (Hausa), en (English)")
user_lang_code = input("Enter language code: ").strip().lower()
user_question = input("Enter your question: ").strip()

final_answer = answer_question(user_question, user_lang_code)
print("\nAnswer:\n", final_answer)

Supported languages: yo (Yoruba), ig (Igbo), ha (Hausa), en (English)

Answer:
 Okay, to, kana so ka san yadda za a kare amfanin gona daga cututtuka? Wannan tambaya ce mai kyau! Kare amfanin gona shine mabuɗin samun girbi mai kyau. Ga wasu shawarwari masu amfani da za ka iya amfani da su nan da nan:

**1. Ka San Abin da Kake Fuskanta:**

*   **Ka gano abin da ke damun:** Kafin ka yi komai, ka yi ƙoƙarin fahimtar abin da ke haifar da matsalar. Shin akwai ɗigo-ɗigo a kan ganyayyaki? Shin tsire-tsire suna lalacewa? Shin dukan gonar ne ko wasu tsire-tsire kaɗan ne kawai? Sanin *abin* da matsalar take zai taimaka maka wajen zaɓar mafita mai kyau. Duba tsire-tsirenka sosai. Shin kana ganin ainihin ƙwaro ko ƙura (alamar *cutar*), ko kuma kawai martanin shukar ne ga cutar (alamar *sakamakon cutar*)?
*   **Shin cuta ce ko wani abu dabam?** Wani lokaci, matsaloli ba cututtuka ba ne kwata-kwata. Wataƙila tsire-tsirenka ba sa samun isasshen ruwa ko abubuwan gina jiki. Ko kuma wataƙila yanayin yana